# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（本問題は単一 DF 入力・単一列出力）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「紹介者の `id` が **2 ではない**顧客、または **未紹介（`referee_id` が欠損）** の顧客の `name` を返せ」
* 入力 DF: `{{INPUT_DATAFRAMES}}`
  `customer: pd.DataFrame` with columns `['id', 'name', 'referee_id']`
* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列は `name` のみ、順序は任意、重複想定なし

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → ベクトル化条件で抽出 → 投影のみ。`referee_id` の **欠損は isna()** で明示、`!= 2` と OR。

```python
import pandas as pd

def select_customers_not_referred_by_2(customer: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        customer (pd.DataFrame): 列 ['id','name','referee_id'] を持つ DataFrame
    Returns:
        pd.DataFrame: 列名と順序は ['name']
    """
    # 列最小化（必要列だけに限定）
    df = customer[['name', 'referee_id']]

    # 条件: referee_id が欠損 もしくは 2 ではない
    # pandas の NA 比較は3値論理のため、isna() を明示し OR を取る
    mask = df['referee_id'].isna() | (df['referee_id'] != 2)

    # 投影: name のみ（順序は任意のため sort はしない）
    out = df.loc[mask, ['name']]

    return out

# Analyze Complexity

# Runtime 28 ms
# Beats 66.31%
# Memory 67.14 MB
# Beats 39.66%

```

### 代替（セミジョイン不要・自己結合不要）

* 本件は単一 DataFrame 内の単純条件なので、`merge` や `map` は不要です。

## 3) アルゴリズム説明

* 使用 API

  * 欠損判定: `Series.isna`
  * ブール条件: ベクトル化比較 `!=` と論理和 `|`
  * 行抽出: `DataFrame.loc[mask, cols]`
  * 列最小化: `[['colA','colB']]` の部分列取り出し
* **NULL / 重複 / 型**

  * `referee_id` が `Int64`（nullable int）でも、`isna()` で欠損を確実に拾い、`| (s != 2)` と組み合わせることで **NULL 安全**に条件を満たす。
  * 出力列は `name` だけ。重複は入力仕様上想定なし（`id` が PK 相当）のため追加処理不要。

## 4) 計算量（概算）

* 列抽出・欠損判定・比較・`loc` はすべてベクトル化で **O(N)**。追加の結合やソートが無い分、最小コスト。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[customer] --> B[列最小化 name, referee_id]
  B --> C["mask = isna(referee_id) OR referee_id != 2"]
  C --> D[loc で抽出]
  D --> E[投影 name のみ]
  E --> F[出力 DataFrame]
```


OK、まだ詰められます。ポイントは「**一発マスク**」「**一時オブジェクト削減**」「**dtype最適化**」です。下の実装は **列コピーを作らず**、`ne(2).fillna(True)` で **NA を True（=採用）に畳み込み**、メモリと時間を少し削ります。

# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、不要な `print` / `sort_values` 禁止

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「紹介者 `referee_id` が **2 以外** または **欠損** の顧客 `name` を返す」
* 入力 DF: `{{INPUT_DATAFRAMES}}`
  `customer: DataFrame`（列 `['id','name','referee_id']`）
* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`
  `name` の 1 列、順序任意

## 2) 実装（指定シグネチャ厳守）

> 一時 DataFrame を作らず、**1本の NA 安全マスク**で抽出 → 最終投影のみ。

```python
import pandas as pd

def select_customers_not_referred_by_2(customer: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        customer (pd.DataFrame): 列 ['id','name','referee_id']
    Returns:
        pd.DataFrame: 列名と順序は ['name']
    """
    # NA を True（採用）として畳み込む → 「≠2 or NA」
    mask = customer['referee_id'].ne(2).fillna(True)

    # 最終投影のみ。中間の列コピーを作らない
    return customer.loc[mask, ['name']]

# Analyze Complexity
# Runtime 296 ms
# Beats 45.99%
# Memory 67.30 MB
# Beats 22.93%

```

### さらに詰める（任意の微最適化）

* **可変長文字の省メモリ化**: `name` が巨大なら PyArrow 文字列を検討（Pandas 2.2+）

  ```python
  # 前処理で1回だけ（ランタイム短縮とメモリ削減に効くことがある）
  # customer['name'] = customer['name'].astype('string[pyarrow]')
  ```
* **整数の nullable 最適化**: `referee_id` を `Int32/Int16` に（NULL を保ちつつメモリ削減）

  ```python
  # 前処理で1回だけ
  # customer['referee_id'] = customer['referee_id'].astype('Int32')
  ```
* **Copy-on-Write**: 既定で有効（2.2）。副作用コピーを抑制でき、中間オブジェクトのコスト低減に寄与。

## 3) アルゴリズム説明

* 使用 API

  * `Series.ne(2).fillna(True)`: NA 安全に「≠ 2 or NA」を 1 式で表現
  * `DataFrame.loc[mask, ['name']]`: 最終投影のみ
* **NULL / 重複 / 型**

  * `ne(2)` は NA を NA のまま返す → `fillna(True)` で **要件に合わせて True 化**
  * 出力重複は仕様上なし（`id` が PK 相当）

## 4) 計算量（概算）

* ブール演算と行抽出のみで **O(N)**。中間 DataFrame を作らないため **メモリと GC の負担が小さめ**。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[customer] --> B["mask = referee_id.ne(2).fillna(True)"]
  B --> C[loc で抽出]
  C --> D[投影 name のみ]
  D --> E[出力 DataFrame]
```

---

### なぜ速く・軽くなるか

* これまでの実装で作っていた **部分列 DataFrame** を廃止 → メモリ削減
* `isna() | (x != 2)` を **`ne(2).fillna(True)` に畳み込み** → 中間 Series の削減
* **投影は最後に 1 回だけ** → 一時配列の生成を抑制

> これ以上の改善は、データ量・分布・環境依存になります。上の微最適化（`string[pyarrow]` / `Int32`）はメモリ 10〜40% 程度改善することがあり、GC/キャッシュヒット向上で **壁時計時間も数〜十数％** 良化することがあります。
